* Efetuar a limpeza do texto, removendo números, underscores, espaços em branco e outros caracteres indesejados; 

* Remover as stop words;

* Executar lematização e stemização;

* Criar uma nuvem de palavras;

* Transformar o dataset para o formato numérico, de tal forma que ele possa ser aplicado em alguma algoritmo de aprendizagem. Para isso você pode utilizar o bag of words e TF-IDF (CountVectorizer e TfidfVectorizer);

* (Desafio – Opcional) Utilizando a saída do tópico anterior, treine um modelo utilizando o algoritmo RandomForestClassifier().



In [33]:
import pandas as pd

In [34]:
df = pd.read_csv("imdb-reviews-pt-br.csv")
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


* Efetuar a limpeza do texto, removendo números, underscores, espaços em branco e outros caracteres indesejados; 

In [35]:
import re

def limpeza(coluna):
    nova_col = coluna\
    .apply(lambda x: re.sub(r'\W+',' ', x))\
    .apply(lambda x: re.sub(r'(\d+)|(\d+\/*\d*)|[_]+','',x))\
    .apply(lambda x: x.lower())
    return nova_col

In [36]:
df['new_text_pt'] = limpeza(df['text_pt'])

In [37]:
df.head()

,id,text_en,text_pt,sentiment,new_text_pt
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este é um exemplo do motivo pelo qual a maiori...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem mesmo os beatles puderam escrever músicas ...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes de fotos de latão não é uma palavra apr...


* Remover as stop words;

In [38]:
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS

In [39]:
#Tokenizando nossas frazes
token_espaco = tokenize.WhitespaceTokenizer()
token_frase = token_espaco.tokenize(df.new_text_pt[0][0:100])
#Limitamos o tamanho da frase [0][0:100] para melhor visualização
print(token_frase)

['mais', 'uma', 'vez', 'o', 'sr', 'costner', 'arrumou', 'um', 'filme', 'por', 'muito', 'mais', 'tempo', 'do', 'que', 'o', 'necessário', 'além', 'das', 'terríve']


In [40]:
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")

frase_processada = list()
for opiniao in df.new_text_pt:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
df["new_text_pt"] = frase_processada

In [41]:
df.head()

,id,text_en,text_pt,sentiment,new_text_pt
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,vez sr costner arrumou filme tempo necessário ...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,exemplo motivo maioria filmes ação mesmos gené...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro tudo odeio raps imbecis poderiam agir...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,beatles puderam escrever músicas todos gostass...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes fotos latão palavra apropriada verdade ...


* Executar lematização e stemização;

In [47]:
# import spacy
# from spacy.cli.download import download
# download(model='pt')

In [43]:
nlp = spacy.load("pt_core_news_sm")

def verifica_lem(words):
    text = ""
    pos = ""
    lemma = ""
    for word in nlp(words):
        text += word.text + "\t"
        pos += word.pos_ + "\t"
        lemma += word.lemma_ + "\t"
    print(text)
    print(pos)
    print(lemma)

In [44]:
verifica_lem(df.new_text_pt[0][0:100])
#Limitamos o tamanho da frase [0][0:100] para melhor visualização

vez	sr	costner	arrumou	filme	tempo	necessário	além	terríveis	seqüências	resgate	mar	quais	poucas	sim	
NOUN	NOUN	NOUN	VERB	NOUN	NOUN	ADJ	ADV	ADJ	NOUN	NOUN	PROPN	DET	PRON	ADV	
vez	sr	costner	arrumar	filme	tempo	necessário	além	terrível	seqüência	resgate	mar	qual	pouco	sim	


In [45]:
def verificar_radical(words):
    
    stemmer = nltk.stem.SnowballStemmer('portuguese')
    text = ""
    stem = ""
    
    for word in token_espaco.tokenize(words):

        text += word + "\t"
        stem += stemmer.stem(word) + "\t"
    
    print(text)
    print(stem)

In [46]:
verificar_radical(df.new_text_pt[0][0:100]) 
#Limitamos o tamanho da frase [0][0:100] para melhor visualização

vez	sr	costner	arrumou	filme	tempo	necessário	além	terríveis	seqüências	resgate	mar	quais	poucas	sim	
vez	sr	costn	arrum	film	temp	necessári	além	terrív	sequênc	resgat	mar	qua	pouc	sim	
